## Final model tuner

This is (hopefully) the final model tuner in this project. It should produce models with an acceptable accuracy for production.

### Please ensure that the data augmentation with suitable augmentation parameters process has been run and there are *train.tfrecord*, *validation.tfrecord* and *test.tfrecord* files in *data/processed*. 

In [1]:
from os.path import join
import tensorflow as tf

from src.image_handling import decode_record

processed = join('data', 'processed')

features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}
shape = (32, 32, 1)

train_dataset = decode_record(join(processed, 'train.tfrecord'), features, shape)
validation_dataset = decode_record(join(processed, 'validation.tfrecord'), features, shape)
test_dataset = decode_record(join(processed, 'test.tfrecord'), features, shape)

In [2]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop, SGD

def create_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(2**hp.Int('2**num_filter_0', 4, 6),
        (4,4) ,activation='relu', input_shape=(32, 32, 1)))

    for i in range(hp.Int('num_cnn_layers', 0, 3)):
        filter = 2**hp.Int('2**num_filter_' + str(i), 4, 7)
        model.add(layers.Conv2D(filter, (4,4), activation='relu', padding='same'))
        if hp.Boolean('pooling_' + str(i)):
            model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        nodes = 2**hp.Int('2**num_nodes_' + str(i), 4, 7)
        model.add(layers.Dense(nodes, activation='relu'))
    
    model.add(layers.Dense(3, 'softmax'))

    optimizers = {
        'adam': Adam(),
        'adadelta': Adadelta(),
        'sgd': SGD(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]),
            momentum=hp.Float('momentum', 0.6, 1, 0.1),
            nesterov=hp.Boolean('nesterov')),
        'rms': RMSprop(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]))
    }

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', list(optimizers.keys()))],
        metrics=['acc'])

    return model

In [3]:
from datetime import datetime
from tensorboard.plugins.hparams import api
from kerastuner import RandomSearch
from tensorflow import summary
from tensorflow.keras.callbacks import TensorBoard

class customTuner(RandomSearch):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def run_trial(self, trial, *args, **kwargs):
        callbacks = kwargs.pop('callbacks', [])
        callbacks = self._deepcopy_callbacks(callbacks)
        trial_dir = self.get_trial_dir(trial.trial_id)
        callbacks.append(TensorBoard(trial_dir))
        kwargs['callbacks'] = callbacks
        super().run_trial(trial, *args, **kwargs)

    def on_trial_end(self, trial):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the hparams where the metrics of tensorboard are
        hparam_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        hparams = trial.hyperparameters.values
        with summary.create_file_writer(hparam_dir).as_default():
            api.hparams(hparams, trial_id=trial.trial_id)

        print(datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
        print('Remaining Trials: ' + str(self.remaining_trials))
        
        super().on_trial_end(trial)

    def on_epoch_end(self, trial, model, epoch, logs):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the data where the metrics of tensorboard are
        hist_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        with summary.create_file_writer(hist_dir).as_default():
            for layer in model.weights:
                summary.histogram(layer.name, data=layer, step=epoch)
        super().on_epoch_end(trial, model, epoch, logs)

In [4]:
from kerastuner import HyperParameters

hp=HyperParameters()
log_dir = join('logs', 'srp18')
timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

tuner = customTuner(
    create_model,
    hyperparameters=hp,
    objective='acc',
    max_trials=1000,
    executions_per_trial=1,
    directory=log_dir,
    project_name=timestamp)

tuner.search_space_summary()

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping(monitor='loss', patience=3) ]

tuner.search(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    steps_per_epoch=100,
    validation_steps=100,
    verbose=0,
    callbacks=callbacks)

2019-11-20T04-13-36
Remaining Trials: 999
2019-11-20T04-14-07
Remaining Trials: 998
2019-11-20T04-14-37
Remaining Trials: 997
2019-11-20T04-15-15
Remaining Trials: 996
2019-11-20T04-15-41
Remaining Trials: 995
2019-11-20T04-15-49
Remaining Trials: 994
2019-11-20T04-16-22
Remaining Trials: 993
2019-11-20T04-17-00
Remaining Trials: 992
2019-11-20T04-17-32
Remaining Trials: 991
2019-11-20T04-18-28
Remaining Trials: 990
2019-11-20T04-19-10
Remaining Trials: 989
2019-11-20T04-19-41
Remaining Trials: 988
2019-11-20T04-20-12
Remaining Trials: 987
2019-11-20T04-20-20
Remaining Trials: 986
2019-11-20T04-20-47
Remaining Trials: 985
2019-11-20T04-21-34
Remaining Trials: 984
2019-11-20T04-22-01
Remaining Trials: 983
2019-11-20T04-22-25
Remaining Trials: 982
2019-11-20T04-22-46
Remaining Trials: 981
2019-11-20T04-23-22
Remaining Trials: 980
2019-11-20T04-23-47
Remaining Trials: 979
2019-11-20T04-24-14
Remaining Trials: 978
2019-11-20T04-25-01
Remaining Trials: 977
2019-11-20T04-25-08
Remaining Tria

In [0]:
tuner.results_summary()